In [ ]:
# %pip install cython
# %pip install sortedcontainers
# %pip install zipp==3.1.0 --upgrade
# %pip install scorer

In [ ]:

import sys

import pyximport
import scorer
import pystan
import os
import numpy as np
import pandas as pd
import lightgbm as lg
import catboost
import zipp
from catboost.utils import get_gpu_device_count
from tqdm import tqdm
from pathlib import Path


# Если запускаемся на DataSphere
if str(Path.home()) == "/home/jupyter":
    os.chdir("/home/jupyter/work/resources/2020wunder_summer/examples/ml_example")
    sys.path.append("/home/jupyter/work/resources/2020wunder_summer/scorer")

print("curdir = ", Path.cwd())
print("homedir = ", Path.home())


sys.path.append("/home/gpu/wunder_summer/scorer")

sys.path.append("../../scorer/")
# sys.path.append("../../scorer/orderbook_fast")
# sys.path.append("/home/gpu/wunder_summer/scorer/orderbook_fast")

# import orderbook as ob
# Чтобы использовать быстрый ордербук раскомментируйте строку:
import orderbook_fast as ob

SIDE_BID = 0
SIDE_ASK = 1

## Собираем датасет для тренировки модели

In [ ]:
%time

def get_simple_features_from_orderbook(orderbook, max_index=2):
    '''
        Считаем простые фичи по ордербуку:
    '''
    spread = orderbook.get_best_price(SIDE_ASK) - orderbook.get_best_price(SIDE_BID)
    orderbook_features = [spread, np.sqrt(spread), spread**2]


    for side in (SIDE_BID, SIDE_ASK):
        for ix in range(max_index):
            price_level = orderbook.get_price_level_at_ix(side, ix)
            if price_level is None:
                orderbook_features += [-1, -1]
            else:
                orderbook_features += [price_level.get_volume(),
                             price_level.get_num_orders()]
    return orderbook_features


def get_simple_deals_features(last_deals, orderbook):
    '''
        Считаем простые фичи по последним сделкам:
    '''
    cur_mean_price = orderbook.get_mean_price()
    cur_time = orderbook.get_time()

    deal_features = []
    for side in (SIDE_BID, SIDE_ASK):
        deal_event = last_deals[side]
        if deal_event is None:
            deal_features += [-1e9, -1e9, -1e9]
        else:
            deal_features += [cur_mean_price - deal_event.price,
                         cur_time - deal_event.time,
                         deal_event.amount]
    return deal_features

def collect_dataset(data_path):
    '''
        Собираем датасет
    '''
    event_player = ob.EventPlayer(data_path)
    orderbook = ob.OrderBook()

    X = []
    Y = []

    last_deals = [None, None]
    for ev in tqdm(event_player.iget_events(),
                    total=len(event_player),
                    desc="collecting dataset"):
        if ev.action == ob.Action.DEAL:
            last_deals[ev.side] = ev
        elif ev.action == ob.Action.NEW_CHUNK:
            last_deals = [None, None]

        orderbook.apply_event(ev)
        if ev.need_prediction:
            features = get_simple_features_from_orderbook(orderbook, max_index=13)
            features += get_simple_deals_features(last_deals, orderbook)

            X.append(features)
            Y.append(ev.Y)

    print(f"Dataset collected: len(X) = {len(X)}")
    return np.array(X), np.array(Y)


# X_train, Y_train = collect_dataset("../../data/train_small_A.npz")
X_train, Y_train = collect_dataset("../../data/train.npz")
X_test, Y_test = collect_dataset("../../data/train_small_B.npz")


## Обучаем модель градиентного бустинга

In [ ]:
%%time
def train_classifier(X_train, Y_train, X_test, Y_test):
    '''
        Обучаем классификатор
    '''

    clf = lg.LGBMClassifier(num_leaves=31, n_estimators=10, learning_rate=0.1)
    clf.fit(X_train, Y_train, eval_set=[(X_test, Y_test)],
            eval_metric="auc", early_stopping_rounds=20)
    return clf

# clf = train_classifier(X_train, Y_train, X_test, Y_test)

In [ ]:
#!L
train_pool = catboost.Pool(X_train, Y_train)

clf = catboost.CatBoostClassifier(
    learning_rate=0.02,
    depth=6,
    l2_leaf_reg=2,

    bootstrap_type="Bayesian",
    # scale_pos_weight=en.scale_pos_weight,
    # scale_pos_weight=1,
    loss_function = "Logloss",
    eval_metric="AUC:hints=skip_train~false",
    custom_metric=["Recall", "Precision", "Accuracy", "F1", "Kappa", "MCC"],
    # use_best_model=True,
    iterations=2000,
    od_type="Iter",
    od_wait=30,
    # rsm=0.5,
    # random_seed=100,
    task_type="GPU",
    boosting_type='Ordered',
    logging_level="Verbose",
    train_dir="grid",

)


clf.fit(
        X=train_pool,
        plot=False      # PLOT,

    )


In [ ]:
#!L

def train_catboost(X_train, Y_train, X_test, Y_test):

    task_type = "GPU" #  if (get_gpu_device_count() == 0) else "GPU"
    print(f"catboost training with {task_type}...")
    print("X_train.shape = ", X_train.shape)
    # print("get_gpu_device_count() = ", get_gpu_device_count())

    train_pool = catboost.Pool(X_train, Y_train)

    grid = {'learning_rate': [0.03, 0.01],
            # 'score_function': ["Cosine", "L2", "NewtonL2"],
            'depth': [4, 5, 6],
            'l2_leaf_reg': [2, 3, 5, 8, 11],
            # 'nan_mode': ["Min", "Max"],
            # 'fold_len_multiplier': np.linspace(1.1, 3, 3),
            # 'bagging_temperature': np.linspace(1, 3000, 3),

    }

    model = catboost.CatBoostClassifier(

        bootstrap_type="Bayesian",
        # scale_pos_weight=en.scale_pos_weight,
        # scale_pos_weight=1,
        loss_function = "Logloss",
        eval_metric="AUC:hints=skip_train~false",
        custom_metric=["Recall", "Precision", "Accuracy", "F1", "Kappa", "MCC"],
        # use_best_model=True,
        iterations=100,
        od_type="Iter",
        od_wait=30,
        # rsm=0.5,
        # random_seed=100,
        task_type=task_type,
        boosting_type='Ordered',
        logging_level="Verbose",
        train_dir="grid",

    )

    grid_search_result = model.grid_search(
        grid,
        X=train_pool,
        stratified=True,
        cv=3,
        search_by_train_test_split=False,
        plot=False      # PLOT,

    )


    params = model.get_params()
    for param in params.items():
        print(param)

    return grid_search_result, model

grid_search_result, clf = train_catboost(X_train, Y_train, X_test, Y_test)
# clf = train_classifier(X_train, Y_train, X_test, Y_test)

## Тестируем получившийся классификатор

In [ ]:
import pystan

In [ ]:
# %pip install cmake
# %pip install scorer==2.0.2 --upgrade --use-feature=2020-resolver --force-reinstall
%pip install pystan


In [ ]:
last_deals = [None, None]

def process_event_and_predict_proba(event, orderbook):
    if event.action == ob.Action.DEAL:
        last_deals[event.side] = event
    elif event.action == ob.Action.NEW_CHUNK:
        last_deals[:] = [None, None]

    if not event.need_prediction:
        return None

    features = get_simple_features_from_orderbook(orderbook, max_index=13)
    features += get_simple_deals_features(last_deals, orderbook)

    proba = clf.predict_proba(features)[1]

    # proba = clf.predict_proba([features_])[0, 1]
    return proba

from scorer import Scorer

scoring = Scorer("../../data/train_small_C.npz")
roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

## Сохраним модель, и решение для отправки готово

In [ ]:
# Сохраним нашу модель
# clf.booster_.save_model("wunder.model")
print(Path.cwd())
clf.save_model("wunder.model", format="cbm")

'''Посмотрите код файла solution.py.
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю.
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''